In [3]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
uk_emb = KeyedVectors.load_word2vec_format("./databases/cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("./databases/cc.ru.300.vec")
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [29]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [30]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

In [31]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf_8_sig') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [32]:
uk_ru_train, X_train, Y_train = load_word_pairs("./text/ukr_rus.train.txt")

In [33]:
uk_ru_test, X_test, Y_test = load_word_pairs("./text/ukr_rus.test.txt")

In [34]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression().fit(X_train, Y_train)

In [35]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541592955589294),
 ('июнь', 0.8411963582038879),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.8359214663505554),
 ('февраль', 0.8328747749328613),
 ('октябрь', 0.8311805129051208),
 ('ноябрь', 0.8278146982192993),
 ('июль', 0.823634922504425),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8038000464439392)]

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        for (f, s) in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn):
            if ru == f:
                num_matches += 1
    precision_val = num_matches / len(pairs)
    print(precision_val)
    return precision_val

In [37]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0.0
1.0
1.0


In [38]:
assert precision(uk_ru_test, Y_test) == 1.0

1.0


In [39]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.811

0.6356589147286822
0.8113695090439277


In [ ]:
from numpy.linalg import svd

def learn_transform(X_train, Y_train):
    u, s, vh = svd(np.matmul(X_train.T, Y_train), full_matrices=True)
    return np.matmul(u, vh)

In [41]:
W = learn_transform(X_train, Y_train)

In [42]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])[0]

('апрель', 0.8237907886505127)

In [43]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

0.6537467700258398
0.8242894056847545


In [49]:
with open("./text/fairy_tale.txt", "r", encoding='utf8') as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [50]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def translate(sentence):
    tokens = tokenizer.tokenize(sentence.lower())
    translated_tokens = []

    for tok in tokens:
       
        if tok not in uk_emb:          
            translated_tokens.append(tok)
            continue

        uk_vec = uk_emb[tok]
        mapped_vec = np.matmul(uk_vec, W)
        ru_word = ru_emb.most_similar([mapped_vec], topn=1)[0][0]
        translated_tokens.append(ru_word)

    return " ".join(translated_tokens)

In [51]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [52]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [53]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала избушку , и и живет . а оно приходят морозы . из лисичка замерзла и и побежала во село огня добувать , чтобы витопити . прибегает к одной бабы и и говорит :

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые были , бабушку ! со воскресеньем ... позичте мне огня , мной тебе одслужу .

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — хорошо , — говорит , — лисичко – сестричка . садись погрійся немного , пока мной пирожки повибираю со печи !

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мач